# Balanceo de Clases para Clasificación de Noticias

Este notebook implementa técnicas de balanceo de datos para un dataset de noticias:
- **Undersampling**: Para clases con muchas muestras (Macroeconomía, Alianzas)
- **Oversampling**: Para clases con pocas muestras (Regulaciones, Sostenibilidad, Otra, Reputación)

Al final se obtendra un dataset balanceado con 200 muestras por clase.

In [32]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import random
import os
from typing import List, Tuple, Dict
from gensim.models.doc2vec import Word2Vec, Doc2Vec, TaggedDocument
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

# Configurar semilla para reproducibilidad
np.random.seed(42)
random.seed(42)

----
## 1. Carga y Preparación de Datos

In [33]:
def cargar_datos_originales() -> Tuple[List[List[str]], List[str]]:
    # Cargar documentos tokenizados
    with open('archivos/lemas_por_documento.txt', 'r', encoding='utf-8') as f:
        docs = [eval(line.strip()) for line in f]
    
    # Cargar etiquetas desde el CSV
    df = pd.read_csv('recursos/news.csv')
    etiquetas = df['Type'].tolist()
    
    print(f"Total de documentos cargados: {len(docs)}")
    print(f"Total de etiquetas cargadas: {len(etiquetas)}")
    
    # Verificar que coincidan
    assert len(docs) == len(etiquetas), "El número de documentos y etiquetas no coincide"
    
    return docs, etiquetas

In [35]:
def mostrar_distribucion_clases(etiquetas: List[str]) -> None:
    """
    Muestra la distribución actual de clases.
    """
    df_temp = pd.DataFrame({'Type': etiquetas})
    distribucion = df_temp['Type'].value_counts()
    
    print("Distribución actual de clases:")
    print(distribucion)
    print(f"\nTotal de documentos: {len(etiquetas)}")
    
    return distribucion

In [36]:
# Cargar datos originales
documentos_originales, etiquetas_originales = cargar_datos_originales()
distribucion_original = mostrar_distribucion_clases(etiquetas_originales)

Total de documentos cargados: 1217
Total de etiquetas cargadas: 1217
Distribución actual de clases:
Type
Macroeconomia     340
Alianzas          247
Innovacion        195
Regulaciones      142
Sostenibilidad    137
Otra              130
Reputacion         26
Name: count, dtype: int64

Total de documentos: 1217


---
## 2. Entrenamiento del Modelo Doc2Vec

Entrenamos un modelo Doc2Vec con todos los documentos originales para crear vectorizaciones consistentes.

In [39]:
def entrenar_doc2vec_global(documentos: List[List[str]], vector_size: int = 100) -> Doc2Vec:
    # Preparar documentos etiquetados
    tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(documentos)]
    
    # Entrenar modelo
    model = Doc2Vec(
        tagged_data, 
        vector_size=vector_size, 
        window=10, 
        min_count=1, 
        dm=0, 
        epochs=20, 
        workers=4
    )
    
    # Guardar modelo
    model.save("archivos/doc2vec_model_global")
    print(f"Modelo guardado. Dimensión de vectores: {vector_size}")
    
    return model

In [40]:
# Entrenar modelo global
modelo_doc2vec = entrenar_doc2vec_global(documentos_originales)

Modelo guardado. Dimensión de vectores: 100


---
## 3. Funciones de Balanceo

### 3.1 Undersampling con Near Miss

In [46]:
def aplicar_undersampling(documentos: List[List[str]], etiquetas: List[str], 
                         clases_objetivo: List[str], n_muestras: int,
                         modelo: Doc2Vec) -> Tuple[np.ndarray, List[str]]:

    print(f"Aplicando undersampling a clases: {clases_objetivo}")
    
    # Filtrar documentos de las clases objetivo
    indices_objetivo = [i for i, etiq in enumerate(etiquetas) if etiq in clases_objetivo]
    docs_filtrados = [documentos[i] for i in indices_objetivo]
    etiquetas_filtradas = [etiquetas[i] for i in indices_objetivo]
    
    # Vectorizar documentos
    X = np.array([modelo.infer_vector(doc) for doc in docs_filtrados])
    
    # Configurar estrategia de sampling
    sampling_strategy = {clase: n_muestras for clase in clases_objetivo}
    
    # Aplicar Near Miss
    nm = NearMiss(version=1, sampling_strategy=sampling_strategy)
    X_resampled, y_resampled = nm.fit_resample(X, etiquetas_filtradas)
    
    print(f"Documentos antes del undersampling: {len(docs_filtrados)}")
    print(f"Documentos después del undersampling: {len(X_resampled)}")
    
    # Convertir a lista si es necesario
    if hasattr(y_resampled, 'tolist'):
        y_resampled = y_resampled.tolist()
    
    return X_resampled, y_resampled

### 3.2 Oversampling Híbrido (Word2Vec + SMOTE)

In [47]:
def generar_muestras_word2vec(documentos: List[List[str]], n_muestras: int) -> List[List[str]]:
    if n_muestras <= 0:
        return []
    
    print(f"Generando {n_muestras} muestras con Word2Vec...")
    
    # Entrenar modelo Word2Vec
    modelo_w2v = Word2Vec(documentos, window=10, min_count=1, sg=0, workers=4)
    
    nuevas_muestras = []
    for _ in range(n_muestras):
        # Seleccionar documento base aleatorio
        doc_base = documentos[np.random.randint(0, len(documentos))]
        
        # Generar variación reemplazando palabras (30% probabilidad)
        nueva_muestra = []
        for palabra in doc_base:
            if palabra in modelo_w2v.wv and np.random.random() < 0.3:
                try:
                    similares = modelo_w2v.wv.most_similar(palabra, topn=3)
                    if similares:
                        nueva_palabra = similares[np.random.randint(0, len(similares))][0]
                        nueva_muestra.append(nueva_palabra)
                    else:
                        nueva_muestra.append(palabra)
                except:
                    nueva_muestra.append(palabra)
            else:
                nueva_muestra.append(palabra)
        
        nuevas_muestras.append(nueva_muestra)
    
    return nuevas_muestras

In [48]:
def aplicar_oversampling_hibrido(documentos: List[List[str]], etiqueta: str, 
                               n_muestras_objetivo: int, modelo_doc2vec: Doc2Vec) -> np.ndarray:

    print(f"\nAplicando oversampling híbrido a clase '{etiqueta}'")
    print(f"Muestras actuales: {len(documentos)}")
    print(f"Muestras objetivo: {n_muestras_objetivo}")
    
    if len(documentos) >= n_muestras_objetivo:
        print("Ya tiene suficientes muestras, aplicando solo vectorización")
        return np.array([modelo_doc2vec.infer_vector(doc) for doc in documentos[:n_muestras_objetivo]])
    
    # Calcular muestras necesarias
    muestras_faltantes = n_muestras_objetivo - len(documentos)
    muestras_word2vec = muestras_faltantes // 2
    
    # Generar muestras con Word2Vec
    muestras_nuevas = generar_muestras_word2vec(documentos, muestras_word2vec)
    
    # Combinar documentos originales y nuevos
    documentos_combinados = documentos + muestras_nuevas
    
    # Vectorizar todos los documentos
    X_vectorizado = np.array([modelo_doc2vec.infer_vector(doc) for doc in documentos_combinados])
    
    # Preparar para SMOTE
    y_dummy = np.array([0] * len(documentos) + [1] * len(muestras_nuevas))
    
    # Calcular muestras adicionales necesarias con SMOTE
    muestras_smote = n_muestras_objetivo - len(X_vectorizado)
    
    if muestras_smote > 0:
        # Aplicar SMOTE
        k_neighbors = min(5, len(X_vectorizado) - 1)
        smote = SMOTE(sampling_strategy={1: len(muestras_nuevas) + muestras_smote}, 
                     k_neighbors=k_neighbors, random_state=42)
        X_resampled, _ = smote.fit_resample(X_vectorizado, y_dummy)
        
        # Tomar exactamente las muestras necesarias
        X_final = X_resampled[:n_muestras_objetivo]
    else:
        X_final = X_vectorizado
    
    print(f"Muestras generadas con Word2Vec: {len(muestras_nuevas)}")
    print(f"Muestras adicionales con SMOTE: {max(0, muestras_smote)}")
    print(f"Total final: {len(X_final)}")
    
    return X_final

---
## 4. Aplicación del Balanceo por Clase

### 4.1 Undersampling para clases con muchas muestras

In [49]:
# Aplicar undersampling a Macroeconomía y Alianzas
clases_undersampling = ['Macroeconomia', 'Alianzas']
X_undersampled, y_undersampled = aplicar_undersampling(
    documentos_originales, 
    etiquetas_originales, 
    clases_undersampling, 
    200, 
    modelo_doc2vec
)

print(f"\nResultados undersampling:")
for clase in clases_undersampling:
    count = y_undersampled.count(clase)
    print(f"{clase}: {count} muestras")

Aplicando undersampling a clases: ['Macroeconomia', 'Alianzas']
Documentos antes del undersampling: 587
Documentos después del undersampling: 400

Resultados undersampling:
Macroeconomia: 200 muestras
Alianzas: 200 muestras


c:\Users\naomi\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


### 4.2 Oversampling para clases con pocas muestras

In [50]:
def separar_documentos_por_clase(documentos: List[List[str]], etiquetas: List[str]) -> Dict[str, List[List[str]]]:
    docs_por_clase = {}
    for doc, etiqueta in zip(documentos, etiquetas):
        if etiqueta not in docs_por_clase:
            docs_por_clase[etiqueta] = []
        docs_por_clase[etiqueta].append(doc)
    
    return docs_por_clase

In [53]:
# Separar documentos por clase
docs_por_clase = separar_documentos_por_clase(documentos_originales, etiquetas_originales)

# Clases que necesitan oversampling
clases_oversampling = ['Regulaciones', 'Sostenibilidad', 'Otra', 'Reputacion']

# Aplicar oversampling a cada clase
resultados_oversampling = {}
for clase in clases_oversampling:
    if clase in docs_por_clase:
        print(f"\n--- Procesando clase: {clase} ---")
        vectores_balanceados = aplicar_oversampling_hibrido(
            docs_por_clase[clase], 
            clase, 
            200, 
            modelo_doc2vec
        )
        resultados_oversampling[clase] = vectores_balanceados
    else:
        print(f"Advertencia: Clase '{clase}' no encontrada en los datos")


--- Procesando clase: Regulaciones ---

Aplicando oversampling híbrido a clase 'Regulaciones'
Muestras actuales: 142
Muestras objetivo: 200
Generando 29 muestras con Word2Vec...


c:\Users\naomi\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Muestras generadas con Word2Vec: 29
Muestras adicionales con SMOTE: 29
Total final: 200

--- Procesando clase: Sostenibilidad ---

Aplicando oversampling híbrido a clase 'Sostenibilidad'
Muestras actuales: 137
Muestras objetivo: 200
Generando 31 muestras con Word2Vec...


c:\Users\naomi\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Muestras generadas con Word2Vec: 31
Muestras adicionales con SMOTE: 32
Total final: 200

--- Procesando clase: Otra ---

Aplicando oversampling híbrido a clase 'Otra'
Muestras actuales: 130
Muestras objetivo: 200
Generando 35 muestras con Word2Vec...


c:\Users\naomi\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Muestras generadas con Word2Vec: 35
Muestras adicionales con SMOTE: 35
Total final: 200

--- Procesando clase: Reputacion ---

Aplicando oversampling híbrido a clase 'Reputacion'
Muestras actuales: 26
Muestras objetivo: 200
Generando 87 muestras con Word2Vec...
Muestras generadas con Word2Vec: 87
Muestras adicionales con SMOTE: 87
Total final: 200


c:\Users\naomi\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


---
## 5. Combinación de Resultados y Dataset Final

In [54]:
def crear_dataset_balanceado(X_undersampled: np.ndarray, y_undersampled: List[str],
                           resultados_oversampling: Dict[str, np.ndarray]) -> pd.DataFrame:
    """
    Combina todos los resultados en un dataset balanceado final.
    
    Args:
        X_undersampled: Vectores de clases con undersampling
        y_undersampled: Etiquetas de clases con undersampling
        resultados_oversampling: Diccionario con vectores de clases con oversampling
        
    Returns:
        pd.DataFrame: Dataset balanceado final
    """
    print("Creando dataset balanceado final...")
    
    # Combinar todos los vectores y etiquetas
    vectores_finales = []
    etiquetas_finales = []
    
    # Agregar resultados de undersampling
    vectores_finales.append(X_undersampled)
    etiquetas_finales.extend(y_undersampled)
    
    # Agregar resultados de oversampling
    for clase, vectores in resultados_oversampling.items():
        vectores_finales.append(vectores)
        etiquetas_finales.extend([clase] * len(vectores))
    
    # Combinar todos los vectores
    X_final = np.vstack(vectores_finales)
    
    # Crear DataFrame
    columnas = [f"dim_{i}" for i in range(X_final.shape[1])]
    df_final = pd.DataFrame(X_final, columns=columnas)
    df_final['Type'] = etiquetas_finales
    
    return df_final

In [55]:
# Crear dataset final
dataset_balanceado = crear_dataset_balanceado(X_undersampled, y_undersampled, resultados_oversampling)

# Mostrar resumen del dataset final
print("Dataset balanceado creado:")
print(f"Forma del dataset: {dataset_balanceado.shape}")
print(f"Columnas: {len(dataset_balanceado.columns)} ({len(dataset_balanceado.columns)-1} dimensiones + 1 etiqueta)")

# Mostrar distribución final
print("\nDistribución final de clases:")
distribucion_final = dataset_balanceado['Type'].value_counts()
print(distribucion_final)

# Verificar balanceo
print(f"\nTotal de muestras: {len(dataset_balanceado)}")
print(f"Muestras por clase (objetivo: 200): {distribucion_final.tolist()}")

Creando dataset balanceado final...
Dataset balanceado creado:
Forma del dataset: (1200, 101)
Columnas: 101 (100 dimensiones + 1 etiqueta)

Distribución final de clases:
Type
Alianzas          200
Macroeconomia     200
Regulaciones      200
Sostenibilidad    200
Otra              200
Reputacion        200
Name: count, dtype: int64

Total de muestras: 1200
Muestras por clase (objetivo: 200): [200, 200, 200, 200, 200, 200]


---
## 6. Guardado de Resultados

In [56]:
# Guardar dataset balanceado
dataset_balanceado.to_csv('archivos/dataset_balanceado_final.csv', index=False)
print("Dataset balanceado guardado como 'archivos/dataset_balanceado_final.csv'")

# Guardar también por separado para compatibilidad (opcional)
print("\nGuardando archivos individuales por clase...")
for clase in dataset_balanceado['Type'].unique():
    df_clase = dataset_balanceado[dataset_balanceado['Type'] == clase].drop('Type', axis=1)
    nombre_archivo = f'archivos/datos_balanceados_{clase.lower()}.csv'
    df_clase.to_csv(nombre_archivo, index=False)
    print(f"- {clase}: {nombre_archivo} ({len(df_clase)} muestras)")

Dataset balanceado guardado como 'archivos/dataset_balanceado_final.csv'

Guardando archivos individuales por clase...
- Alianzas: archivos/datos_balanceados_alianzas.csv (200 muestras)
- Macroeconomia: archivos/datos_balanceados_macroeconomia.csv (200 muestras)
- Regulaciones: archivos/datos_balanceados_regulaciones.csv (200 muestras)
- Sostenibilidad: archivos/datos_balanceados_sostenibilidad.csv (200 muestras)
- Otra: archivos/datos_balanceados_otra.csv (200 muestras)
- Reputacion: archivos/datos_balanceados_reputacion.csv (200 muestras)


---
## 7. Verificación y Resumen Final

In [57]:
def verificar_dataset_balanceado(df: pd.DataFrame) -> None:
    """
    Realiza verificaciones finales del dataset balanceado.
    """
    print("=== VERIFICACIÓN FINAL ===")
    print(f"Forma del dataset: {df.shape}")
    print(f"Clases únicas: {df['Type'].nunique()}")
    print(f"Valores nulos: {df.isnull().sum().sum()}")
    
    print("\nDistribución por clase:")
    for clase, count in df['Type'].value_counts().items():
        print(f"  {clase}: {count} muestras")
    
    print(f"\nRango de valores en las dimensiones:")
    dimensiones = [col for col in df.columns if col.startswith('dim_')]
    print(f"  Mínimo: {df[dimensiones].min().min():.4f}")
    print(f"  Máximo: {df[dimensiones].max().max():.4f}")
    print(f"  Media: {df[dimensiones].mean().mean():.4f}")
    
    # Verificar que todas las clases tengan exactamente 200 muestras
    objetivo = 200
    todas_balanceadas = all(count == objetivo for count in df['Type'].value_counts().values)
    print(f"\n¿Todas las clases tienen {objetivo} muestras? {'✓ SÍ' if todas_balanceadas else '✗ NO'}")

In [58]:
verificar_dataset_balanceado(dataset_balanceado)

print("\n" + "="*50)
print("PROCESO COMPLETADO EXITOSAMENTE")
print("="*50)
print(f"Dataset balanceado disponible en: 'archivos/dataset_balanceado_final.csv'")
print("Cada clase tiene exactamente 200 muestras.")
print("El dataset contiene vectores Doc2Vec de 100 dimensiones + columna 'Type' con la etiqueta.")

=== VERIFICACIÓN FINAL ===
Forma del dataset: (1200, 101)
Clases únicas: 6
Valores nulos: 0

Distribución por clase:
  Alianzas: 200 muestras
  Macroeconomia: 200 muestras
  Regulaciones: 200 muestras
  Sostenibilidad: 200 muestras
  Otra: 200 muestras
  Reputacion: 200 muestras

Rango de valores en las dimensiones:
  Mínimo: -3.4970
  Máximo: 3.9888
  Media: 0.0419

¿Todas las clases tienen 200 muestras? ✓ SÍ

PROCESO COMPLETADO EXITOSAMENTE
Dataset balanceado disponible en: 'archivos/dataset_balanceado_final.csv'
Cada clase tiene exactamente 200 muestras.
El dataset contiene vectores Doc2Vec de 100 dimensiones + columna 'Type' con la etiqueta.
